# Positional Embedding

In [1]:
import re
from abc import ABC, abstractmethod
import numpy as np

np.random.seed(99)

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = None
        self.gradient_fn = lambda: None
        self.parents = set()

    def gradient(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.gradient()

    def shape(self):
        return self.data.shape

    def size(self):
        return np.prod(self.data.shape[1:])

    def __add__(self, other):
        p = Tensor(self.data + other.data)

        def gradient_fn():
            self.grad = p.grad
            other.grad = p.grad

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

    def __mul__(self, other):
        p = Tensor(self.data * other.data)

        def gradient_fn():
            self.grad = p.grad * other.data
            other.grad = p.grad * self.data

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

    def concat(self, other, axis):
        p = Tensor(np.concatenate([self.data, other.data], axis=axis))

        def gradient_fn():
            self.grad, other.grad = np.split(p.grad, [self.data.shape[axis]], axis=axis)

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

In [3]:
class DataLoader:

    def __init__(self, filename, batch_size, stride):
        self.filename = filename
        self.batch_size = batch_size
        self.stride = stride

        with open(self.filename, 'r', encoding='utf-8') as f:
            self.text = f.read().lower()

        self.vocabulary = sorted(set(self.split_text(self.text)))
        self.vocabulary.extend(['<|eos|>', '<|unk|>'])
        self.word2index = {word: index for index, word in enumerate(self.vocabulary)}
        self.index2word = {index: word for index, word in enumerate(self.vocabulary)}
        self.tokens = self.encode(self.text)

        self.train()

    @staticmethod
    def split_text(text):
        words = re.split(r'([,.:;?_!"()\']|\s)', text.lower())
        return [t.strip() for t in words if t.strip()]

    def train(self):
        self.features = []
        self.labels = []
        for i in range(0, len(self.tokens) * 9 // 10 - self.batch_size,
                       self.stride):
            self.features.append(self.tokens[i: i + self.batch_size])
            self.labels.append(self.tokens[i + 1: i + self.batch_size + 1])

    def eval(self):
        self.features = []
        self.labels = []
        for i in range(len(self.tokens) * 9 // 10 - self.batch_size + 1,
                       len(self.tokens) - self.batch_size,
                       self.stride):
            self.features.append(self.tokens[i: i + self.batch_size])
            self.labels.append(self.tokens[i + 1: i + self.batch_size + 1])

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

    def encode(self, text):
        words = self.split_text(text)
        words = [word if word in self.word2index else '<|unk|>' for word in words]
        return [self.word2index[word] for word in words]

    def decode(self, tokens):
        text = " ".join([self.index2word[index] for index in tokens])
        return re.sub(r'\s+([,.:;?_!"()\'])', r'\1', text)

In [4]:
class Layer(ABC):

    def __init__(self):
        self.training = True

    def __call__(self, *args):
        return self.forward(*args)

    @abstractmethod
    def forward(self, *args):
        pass

    def parameters(self):
        return []

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

In [5]:
class Embedding(Layer):

    def __init__(self, vocabulary_size, embedding_size, axis=None):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.axis = axis

        self.weight = Tensor(np.random.rand(embedding_size, vocabulary_size) / vocabulary_size)

    def forward(self, x: Tensor):
        weights = self.weight.data.T[x.data]
        p = Tensor(np.sum(weights, axis=self.axis) if self.axis is not None else weights)

        def gradient_fn():
            if self.weight.grad is None:
                self.weight.grad = np.zeros_like(self.weight.data)
            self.weight.grad.T[x.data] += p.grad

        p.gradient_fn = gradient_fn
        p.parents = {self.weight}
        return p

    def parameters(self):
        return [self.weight]

In [6]:
class GPTEmbedding(Layer):

    def __init__(self, vocabulary_size, context_size, embedding_size):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.context_size = context_size
        self.embedding_size = embedding_size

        self.token_embedding = Embedding(self.vocabulary_size, self.embedding_size)
        self.positional_embedding = Embedding(self.context_size, self.embedding_size)

    def forward(self, x: Tensor):
        token = self.token_embedding(x)
        position = self.positional_embedding(Tensor(range(self.context_size)))
        return token + position

In [7]:
class GPT(Layer):

    def __init__(self, vocabulary_size, context_size, embedding_size):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.context_size = context_size
        self.embedding_size = embedding_size

        self.embedding = GPTEmbedding(self.vocabulary_size, self.context_size, self.embedding_size)

    def forward(self, x: Tensor):
        return self.embedding(x)

In [8]:
CONTEXT_SIZE = 4
EMBEDDING_SIZE = 3

In [9]:
dataset = DataLoader('../one-day.txt', CONTEXT_SIZE, 1)

model = GPT(len(dataset.vocabulary), CONTEXT_SIZE, EMBEDDING_SIZE)

print('Embedding weight shape: ', model.embedding.token_embedding.weight.shape())
print('Embedding weight: ', model.embedding.token_embedding.weight.data)
print('Positional weight shape: ', model.embedding.positional_embedding.weight.shape())
print('Positional weight: ', model.embedding.positional_embedding.weight.data)

feature, label = dataset[0]

prediction = model(Tensor(feature))

print('Token shape: ', prediction.shape())
print('Tokens: ', prediction.data)

Embedding weight shape:  (3, 267)
Embedding weight:  [[2.51789722e-03 1.82800899e-03 3.09174222e-03 1.17776733e-04
  3.02640436e-03 2.11841730e-03 1.11469101e-03 1.74890339e-04
  3.71021498e-03 2.55645432e-05 2.88311996e-03 2.79688053e-03
  1.41362898e-03 1.85073952e-03 3.47920746e-03 1.48110129e-03
  3.64777639e-03 1.96410006e-03 3.50610836e-04 3.04609892e-03
  7.92834403e-04 2.07620144e-03 1.09464088e-03 3.05671296e-03
  3.10128302e-03 8.29877797e-04 2.41511124e-03 3.56485475e-04
  1.54180988e-03 3.62791241e-04 5.39367037e-04 7.94742230e-04
  1.78522904e-03 2.90688527e-04 8.80313788e-04 2.45430812e-05
  3.36570857e-03 2.06829373e-03 6.27515460e-04 3.47894451e-03
  2.03029101e-03 1.56398994e-04 1.96583086e-03 2.39753760e-03
  2.99871971e-03 3.12703690e-03 9.46188128e-04 3.62649256e-03
  1.74621024e-03 9.89105505e-04 2.05759707e-03 1.41323516e-03
  7.63027521e-04 3.42869850e-03 3.51751844e-03 3.13668012e-03
  1.62093384e-03 2.68207098e-03 3.18855427e-03 2.96055001e-04
  1.53542523e-03 